In [2]:
import openai
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
openai.api_key = "......................................................"

In [4]:
df = pd.read_csv('listings_master_data.csv')

In [5]:
len(df)

48120

In [17]:
df.head()

,unit_id,@type,name,url,geolatitude,geolongitude,floorSizevalue,floorSizeunitText,numberOfRoomsname,numberOfRoomsvalue,addressaddressCountry,addressaddressRegion,addressaddressLocality,prices,frequency
0,6965011,House,Large Plot | Modern Kitchen | 3 Bedrooms,https://www.bayut.com/property/details-6965011...,25.052004,55.254704,"3,324",SQFT,Bedroom(s),3,UAE,Dubai,Arabian Ranches,320000.0,Yearly
1,6966601,House,Exclusive | Brand New | Ready to Move In,https://www.bayut.com/property/details-6966601...,24.856453,55.158576,"1,512",SQFT,Bedroom(s),3,UAE,Dubai,Dubai South,90000.0,Yearly
2,6308837,House,Enjoy Convenient Life w/ This Semi-Corner Unit,https://www.bayut.com/property/details-6308837...,24.391229,54.496525,"2,716",SQFT,Bedroom(s),4,UAE,Abu Dhabi,Abu Dhabi Gate City (Officers City),180000.0,Yearly
3,6972016,Apartment,Covered Parking | Balcony | Walk-In Wardrobe,https://www.bayut.com/property/details-6972016...,25.193510,55.277629,"1,033",SQFT,Bedroom(s),1,UAE,Dubai,Downtown Dubai,130000.0,Yearly
4,6310043,Apartment,Balcony+3B+3B ∫ Near Al Ittihad Park,https://www.bayut.com/property/details-6310043...,25.345889,55.388806,"1,400",SQFT,Bedroom(s),3,UAE,Sharjah,Al Qasimia,30000.0,Yearly


In [18]:
df.columns

Index(['unit_id', '@type', 'name', 'url', 'geolatitude', 'geolongitude',
       'floorSizevalue', 'floorSizeunitText', 'numberOfRoomsname',
       'numberOfRoomsvalue', 'addressaddressCountry', 'addressaddressRegion',
       'addressaddressLocality', 'prices', 'frequency'],
      dtype='object')

In [14]:
len(df.columns)

15

In [7]:
df["addressaddressRegion"].unique()

array(['Dubai', 'Abu Dhabi', 'Sharjah', 'Ajman', 'Ras Al Khaimah',
       'Al Ain', 'Umm Al Quwain', 'Al Napoca', 'Fujairah'], dtype=object)

In [13]:
dic = {'region' : [], 
      "number of places" : []}
for region in df["addressaddressRegion"].unique() :
    dic['region'].append(region)
    oussama = df[df["addressaddressRegion"] == region]["addressaddressLocality"].unique()
    dic['number of places'].append(len(oussama))
dic = pd.DataFrame(dic)
dic

,region,number of places
0,Dubai,115
1,Abu Dhabi,80
2,Sharjah,83
3,Ajman,30
4,Ras Al Khaimah,27
5,Al Ain,38
6,Umm Al Quwain,7
7,Al Napoca,1
8,Fujairah,7


In [4]:
df = df.sample(1000 , random_state = 42)

In [5]:
df.head()

,unit_id,@type,name,url,geolatitude,geolongitude,floorSizevalue,floorSizeunitText,numberOfRoomsname,numberOfRoomsvalue,addressaddressCountry,addressaddressRegion,addressaddressLocality,prices,frequency
24632,6878572,Residence,2 BHK Amazing Townhouse | Sea View| Bermuda,https://www.bayut.com/property/details-6878572...,25.718389,55.829181,"2,184",SQFT,Bedroom(s),2,UAE,Ras Al Khaimah,Mina Al Arab,90000.0,Yearly
36428,6934325,Apartment,BRAND NEW !! FIRST TENANT 2BHK NEAR KHALIFA UN...,https://www.bayut.com/property/details-6934325...,24.446993,54.385729,"1,400",SQFT,Bedroom(s),2,UAE,Abu Dhabi,Al Mushrif,51999.0,Yearly
42150,6853128,Apartment,Ready to Move in | Well Maintained | En-suite ...,https://www.bayut.com/property/details-6853128...,25.193889,55.263788,"1,415",SQFT,Bedroom(s),2,UAE,Dubai,Business Bay,139999.0,Yearly
23885,6948948,House,4 CHQS | Move in Ready | Double Row,https://www.bayut.com/property/details-6948948...,24.457580,54.675561,"2,858",SQFT,Bedroom(s),4,UAE,Abu Dhabi,Al Reef,140000.0,Yearly
2174,6665596,House,Luxury | Elegant Furniture| Brand New | Spacious,https://www.bayut.com/property/details-6665596...,25.178825,55.574569,"6,800",SQFT,Bedroom(s),5,UAE,Dubai,Al Awir,199999.0,Yearly


# Start preparing text _ embedding Data

In [50]:
def text_column(data_frame, n) :
    
    col_text = {'text' : [] }
    
    rate = 0.1*len(data_frame) # 10% pourcent of data = 100
    
    for index in range(0 + int( rate )*n , int( rate )*(n+1) ) :
        query = f'take this ligne of data frame that content 15 information and transform it into a understanding information for our client, please try to create this in one paragraphe: this is the ligne of data {data_frame.iloc[index]} '
        response_text = openai.ChatCompletion.create(
        messages=[{'role': 'system', 'content': ''},{'role': 'user', 'content': query}],model='gpt-3.5-turbo',
            temperature=0 ,  request_timeout=120000)
        message = response_text['choices'][0]['message']['content']
        col_text['text'].append(message)
        
    return pd.DataFrame(col_text)



In [23]:
n  = (len(df)/int(len(df)*0.1)) - 1
n

9.0

In [47]:
text_column_1 = text_column(data_frame = df , n = 0)

In [51]:
text_column_2 = text_column(data_frame = df , n = 1)

In [53]:
text_column_3 = text_column(data_frame = df , n = 2)

In [55]:
text_column_4 = text_column(data_frame = df , n = 3)

In [57]:
text_column_5 = text_column(data_frame = df , n = 4)

In [59]:
text_column_6 = text_column(data_frame = df , n = 5)

In [61]:
text_column_7 = text_column(data_frame = df , n = 6)

In [63]:
text_column_8 = text_column(data_frame = df , n = 7)

In [65]:
text_column_9 = text_column(data_frame = df , n = 8)

In [67]:
text_column_10 = text_column(data_frame = df , n = 9)

In [71]:
def embedding_text(row):
    
    response = openai.Embedding.create(input = row['text']  ,  model = "text-embedding-ada-002")
    embed = response['data'][0]['embedding']
    
    return embed
        

In [72]:
def embedding_column(data_frame):
    
    data_frame['embedding'] = data_frame.apply(embedding_text , axis = 1)
    
    return data_frame

In [73]:
embedding_column_1 = embedding_column(text_column_1)

In [75]:
embedding_text_1 = embedding_column_1

In [77]:
embedding_text_2 = embedding_column(text_column_2)

In [79]:
embedding_text_3 = embedding_column(text_column_3)

In [80]:
embedding_text_4 = embedding_column(text_column_4)

In [81]:
embedding_text_5 = embedding_column(text_column_5)

In [82]:
embedding_text_6 = embedding_column(text_column_6)

In [83]:
embedding_text_7 = embedding_column(text_column_7)

In [84]:
embedding_text_8 = embedding_column(text_column_8)

In [85]:
embedding_text_9 = embedding_column(text_column_9)

In [86]:
embedding_text_10 = embedding_column(text_column_10)

In [336]:
df = pd.concat([embedding_text_1,embedding_text_2,embedding_text_3,embedding_text_4,embedding_text_5,
                embedding_text_6,embedding_text_7,embedding_text_8,embedding_text_9,embedding_text_10] , ignore_index=True )

# Final Data

In [346]:
df.head(1000)

,text,embedding
0,We have a residence property available for our...,"[0.0124435443431139, 0.006698851473629475, -0...."
1,We have a brand new 2-bedroom apartment availa...,"[0.02008100226521492, 0.005562980193644762, -0..."
2,We have a ready-to-move-in apartment located i...,"[0.013784174807369709, -0.00345245492644608, -..."
3,This data represents a house located in Al Ree...,"[0.010833458974957466, 0.01742301508784294, -0..."
4,We have a luxurious and elegant house with bra...,"[0.008466669358313084, 0.02172725275158882, -0..."
...,...,...
995,We have a vacant 2-bedroom apartment with a co...,"[0.010188404470682144, 0.0012790912296622992, ..."
996,We have a two-bedroom apartment available for ...,"[0.020755285397171974, 0.012024989351630211, -..."
997,We have a very well-maintained apartment with ...,"[0.02788640558719635, 0.0090120704844594, -0.0..."
998,We have a very spacious and luxury 3bhk villa ...,"[0.014151254668831825, 0.00045599567238241434,..."


In [339]:
df_apply = df.copy()

# Chatbot Model

In [362]:
def calcul_similarity(query , row) :
    
    response = openai.Embedding.create(input = query  ,  model = "text-embedding-ada-002")
    embed_query = response['data'][0]['embedding']
    
    vector1 = np.array(row['embedding'])
    vector2 = np.array(embed_query)
    similarity_score = cosine_similarity([vector1], [vector2])[0][0]
    
    return similarity_score


def similarity_column(query): #create a column that content similarities between the vector of text and vector of query
    
    df['similarity'] = df.apply(lambda row : calcul_similarity(query , row) , axis = 1)
    
    return df


def best_answers(query , row , Temperature): #Temperature determine the degree of the variance from his query for which the client desire to generate his response his shortlist
                                             #high Temperature implied multiple choice answer low Temperature implied a restreint shortlist (look up for more accuracy compared to the query) 
    docs = []
    data_frame = similarity_column(query)
    M = max(data_frame['similarity'].unique()) 
    m = min(data_frame['similarity'].unique())
    rate = 1 - Temperature
    P = rate*(M-m)                      Thr = m + (1 - Temperature)*(M-m)
    Thr = m + P  #threshold 
    
    for index , row in data_frame.iterrows() :
        if Thr <= row['similarity'] <= M : #tolerance interval
            docs.append(row['text'])
        
    return docs #content the best responses respecting this accuracy and this tolerance interval 
[]

def ask(query , Temperature) :
    
    if 0 <= Temperature <= 1 :
        
        message = best_answers(query , row , Temperature)
        
        
        return message
    
    else :
        print('KeyError : Temperature is a float number between 0 and 1 (0 <= Temperature <= 1 )')
        
        
    

    
        
    



In [363]:
ask('look up for an apartment in Abu Dhabi UAE region which number of bedrooms between 3 and 5' , 0.1)


['We have a great apartment available for our client with the following details: It has 3 bedrooms, a maid room, and great facilities. The apartment is located on Electra Street in Abu Dhabi, UAE. The floor size is 2,000 square feet. The price for this apartment is 124,995 AED per year. You can find more information and pictures on our website: [link to the property](https://www.bayut.com/property/details-6914752...).',
 "We have a brand new apartment located in Al Reem Island, Abu Dhabi, UAE. This apartment has 3 bedrooms and a maid's room, with a total floor size of 1,705 square feet. The rental price for this apartment is AED 110,000 per year. You can find more details and contact information on the property listing page: [link](https://www.bayut.com/property/details-6951936...).",
 'We have a great apartment available for our client. It is located close to the Corniche in Abu Dhabi, UAE. The apartment has 3 bedrooms and a balcony, making it a comfortable living space. The total flo